In [ ]:
import neural_network as nn
from random import random, uniform
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from sqlalchemy import create_engine

# Credentials to connect to the database
username = "username"
password = "DB_password"
hostname = "DB_host"
dbname = "DB_name"

class Particle:
    def __init__(self, position, velocity):
        self.position = position
        self.velocity = velocity
        self.pbest_position = position
        self.gbest_position = position
        self.pbest_fitness = float('-inf')
        self.gbest_fitness = float('-inf')

    def evaluate(self, obj_function, steady, cost, model):
        fitness = obj_function(self.position, steady, cost, model)
        if fitness > self.pbest_fitness:
            self.pbest_fitness = fitness
            self.pbest_position = self.position
        if fitness > self.gbest_fitness:
            self.gbest_fitness = fitness
            self.gbest_position = self.position

    def update_velocity(self, w, c1, c2):
        r1 = random()
        r2 = random()
        cognitive = c1 * r1 * (self.pbest_position - self.position)
        social = c2 * r2 * (self.gbest_position - self.position)
        self.velocity = (w * self.velocity) + cognitive + social

    def update_position(self, bounds):
        self.position += self.velocity
        self.position = np.clip(self.position, bounds[0], bounds[1])

def particle_swarm(number_of_particles, c1, c2, w_min, w_max, iterations, obj_function, steady, cost, model, bounds):
    swarm = [Particle(position=round(uniform(bounds[0], bounds[1]), 2),
                      velocity=round(uniform(-0.5, 0.5), 2)) for _ in range(number_of_particles)]

    for i in range(iterations):
        w = w_max - i * ((w_max - w_min) / iterations)
        for particle in swarm:
            particle.evaluate(obj_function, steady, cost, model)
            particle.update_velocity(w, c1, c2)
            particle.update_position(bounds)

    best_particle = max(swarm, key=lambda p: p.gbest_fitness)
    return best_particle.gbest_position, best_particle.gbest_fitness

def gain(price, steady, cost, model):
    temp = np.append(steady, price).reshape(1, -1)
    quantity = np.round(model.predict(temp)).astype(int)
    result = (price - cost) * quantity
    return float(np.round(result, 2))

def optimize_prices():
    try:
        engine = create_engine(f"mysql+mysqlconnector://{username}:{password}@{hostname}/{dbname}")
        data = pd.read_sql_table("pso_data", engine)
        opt_data = pd.read_sql_table("data_for_optimization", engine)

        product_encoder = nn.nn_final_training()
        data["product_id"] = product_encoder.transform(data["product_id"].astype(str))
        model = load_model("final_model.h5")

        columns = ["product_id", "low_bound", "high_bound", "optimized_price", "predicted_quantity", "predicted_gain", "arxikiTimi", "telikiTimi"]
        optimization_results = pd.DataFrame(columns=columns)

        for index, row in data.iterrows():
            print(index)
            upper_bound = row.product_max_bound
            lower_bound = row.product_min_bound
            steady = row[2:-1].values
            cost = row.product_cost
            bounds = (lower_bound, upper_bound)
            number_of_swarms = 40
            c1 = 0.4
            c2 = 0.4
            w_min = 0.6
            w_max = 0.8

            best_position, best_value = particle_swarm(number_of_swarms, c1, c2, w_min, w_max, 400, gain, steady, cost, model, bounds)
            temp = np.append(steady, best_position).reshape(1, -1)
            predicted_quantity = int(np.round(model.predict(temp)).astype(int))

            optimization_results.loc[index, ["product_id", "low_bound", "high_bound", "optimized_price", "predicted_quantity", "predicted_gain"]] = [
                row["product_id"], float(bounds[0]), float(bounds[1]), float(round(best_position, 2)), predicted_quantity, round(best_value, 2)
            ]

        optimization_results["product_id"] = product_encoder.inverse_transform(data["product_id"].astype(int))
        optimization_results.to_sql(name="optimization_results", con=engine, index=False, if_exists="replace", chunksize=1)

        for index, row in optimization_results.iterrows():
            arxikiTimi = round(float(opt_data.loc[opt_data["product_id"] == row["product_id"], "arxikiTimi"].values[0]), 2)
            telikiTimi = round(float(opt_data.loc[opt_data["product_id"] == row["product_id"], "telikiTimi"].values[0]), 2)
            optimization_results.loc[index, ["arxikiTimi", "telikiTimi"]] = [arxikiTimi, telikiTimi]

        optimization_results.to_sql(name="optimization_results", con=engine, index=False, if_exists="replace", chunksize=1)

    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    optimize_prices()
